## SIMCLR Implementation


### Import Libraries

In [ ]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
import pickle
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from PIL import Image
# from torchvision import datasets, transforms, models
# from torch.optim.lr_scheduler import StepLR

import numpy as np
import pandas as pd

# import shutil, time, os, requests, random, copy

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.manifold import TSNE
from torchlars import LARS
# from pytorch_metric_learning import losses
from lightly.loss import NTXentLoss

# move one directory up to import from src
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")

# from src.setdevice import set_device

### Set Seeds

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
set_seed()

In [ ]:
def set_device():
    if torch.cuda.is_available():  # check if NVIDIA GPU is available
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():  # check if Apple's Metal is available
        device = torch.device("mps")
        print(
            f"Checking pytorch is built with mps activated: {torch.backends.mps.is_built()}"
        )
    else:
        device = torch.device("cpu")

    if device == torch.device("cuda") or device == torch.device("mps"):
        print(f"Running on {device} GPU...")
    else:
        print("Running on CPU...")

    return device
device = set_device()

Running on cuda GPU...


### Define f(·) encoder - ResNet-20


In [ ]:
"""
It is the neural network model base encoder f(·) that extracts representation vectors 
from augmented data examples. We choose to use the ResNet20 architecture.
"""

# Residual Block
class BasicBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=inplanes,
            out_channels=planes,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, out_channels=planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        if inplanes > planes or stride > 1:
            # Option B
            self.downsample = nn.Sequential(
                nn.Conv2d(
                    in_channels=inplanes,
                    out_channels=planes,
                    kernel_size=1,
                    stride=stride,
                    padding=0,
                    bias=False,
                ),
                nn.BatchNorm2d(planes),
            )
        else:
            self.downsample = None
        self.stride = stride

    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity if self.downsample is None else self.downsample(identity)
        out = F.relu(out)
        return out


# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.inplanes = 16
        self.conv0 = nn.Conv2d(
            in_channels=3,
            out_channels=self.inplanes,
            kernel_size=3,
            stride=1,
            padding=1,
            bias=False,
        )
        self.bn0 = nn.BatchNorm2d(self.inplanes)
        self.layer1 = self._make_layer(block, planes=16, blocks=layers[0], stride=1)
        self.layer2 = self._make_layer(block, planes=32, blocks=layers[1], stride=2)
        self.layer3 = self._make_layer(block, planes=64, blocks=layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # self.identity = nn.Identity()
        # self.fc = nn.Linear(in_features=64, out_features=num_classes)

    def _make_layer(self, block, planes, blocks, stride):
        layers = []
        for i in range(blocks):
            layers.append(
                block(
                    inplanes=(self.inplanes if i == 0 else planes),
                    planes=planes,
                    stride=stride if i == 0 else 1,
                )
            )
        self.inplanes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn0(self.conv0(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        # x = self.identity(x)
        x = x.view(x.size(0), -1)
        # x = self.fc(x)
        # x = self.identity(x)
        return x


### Define g(·) projection head - MLP with linear layers

In [ ]:
"""
It is the neural network projection head g(·) that maps representations to 
the space where contrastive loss is applied. There is the option to use a linear
projection head or a nonlinear projection head (2 layer MLP).
"""

class LinearLayer(nn.Module):
    def __init__(self, in_features, out_features, use_bias=False, use_bn=False):
        super(LinearLayer, self).__init__()

        self.use_bn = use_bn
        self.linear = nn.Linear(
            in_features=in_features,
            out_features=out_features,
            bias=use_bias and not use_bn,
        )
        self.bn = nn.BatchNorm1d(out_features)

    def forward(self, x):
        x = self.linear(x)
        x = self.bn(x) if self.use_bn else x
        return x


class ProjectionHead(nn.Module):
    def __init__(
        self,
        in_features,
        hidden_features,
        out_features,
        head_type="nonlinear",
        use_bn=True,
    ):
        super(ProjectionHead, self).__init__()

        if head_type == "linear":
            self.layers = LinearLayer(
                in_features=in_features,
                out_features=out_features,
                use_bias=False,
                use_bn=True,
            )
        elif head_type == "nonlinear":
            self.layers = nn.Sequential(
                LinearLayer(
                    in_features=in_features,
                    out_features=hidden_features,
                    use_bias=True,
                    use_bn=use_bn,
                ),
                nn.ReLU(),
                LinearLayer(
                    in_features=hidden_features,
                    out_features=out_features,
                    use_bias=False,
                    use_bn=use_bn,
                ),
            )

    def forward(self, x):
        x = self.layers(x)
        return x


### Model

In [ ]:
class SimCLR(nn.Module):
    """
    Creates SimCLR model given encoder
    Args:
      encoder (nn.Module): Encoder
      projection_n_in (int): Number of input features of the projection head
      projection_n_hidden (int): Number of hidden features of the projection head
      projection_n_out (int): Number of output features of the projection head
      projection_use_bn (bool): Whether to use batch norm in the projection head
    """

    def __init__(
        self,
        encoder: nn.Module = ResNet(BasicBlock, [3, 3, 3]),
        projection_n_in: int = 64, # TODO CHECK ON THIS
        projection_n_hidden: int = 64,  # TODO CHECK ON THIS
        projection_n_out: int = 128,
        projection_use_bn: bool = True,
    ):
        super().__init__()

        self.encoder = ResNet(BasicBlock, [3, 3, 3])
        self.projection = ProjectionHead(
            in_features=projection_n_in,
            hidden_features=projection_n_hidden,
            out_features=projection_n_out,
            head_type="nonlinear",
            use_bn=projection_use_bn,
        )

    def forward(self, x):
        feature = self.encoder(x)
        x = self.projection(feature)
        return F.normalize(feature, dim=-1), x


In [ ]:
model = SimCLR().to(device)

### Pre-processing functions for augmenting images

In [ ]:
s = 0.5
train_transform = transforms.Compose(
            [
                transforms.RandomResizedCrop(
                    (32, 32),
                    scale=(0.08, 1.0),
                    ratio=(0.75, 1.3333333333333333),
                ),
                transforms.RandomHorizontalFlip(),
                transforms.RandomApply(
                    [transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)], p=0.8
                ),
                transforms.RandomGrayscale(p=0.2),
                # transforms.RandomApply([transforms.GaussianBlur(kernel_size = 3, sigma=(0.1, 2.0))], p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
            ]
        )

test_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
            ]
        )

### Setting up hyperparameters

In [ ]:
model = SimCLR().to(device)

In [ ]:
# Tunable hyperparameters
LRS = [0.5, 1, 1.5]
TEMPS = [0.1, 0.5, 1.0]
BATCHES = [256, 512, 1024, 2048, 4096]

# for now
# LR = 0.5
TEMP = 0.5
BATCH = 1024
LR = 0.01 * 0.3*BATCH/256 #0.075 * np.sqrt(BATCH)
EPOCHS = 100
DECAY = 1e-6

#############################################
# loss function
criterion = NTXentLoss(temperature=TEMP)

# Add optimizer
optimizer = LARS(optim.Adam(model.parameters(), lr=LR, weight_decay=DECAY))
# optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=DECAY)

# "decay the learning rate with the cosine decay schedule without restarts"
# SCHEDULER OR LINEAR EWARMUP
warmupscheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lambda epoch: (epoch + 1) / 10.0, verbose=True
)

# # SCHEDULER FOR COSINE DECAY
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, 500, eta_min=0.05, last_epoch=-1, verbose=True
)
#############################################

Adjusting learning rate of group 0 to 1.2000e-03.
Epoch 00000: adjusting learning rate of group 0 to 1.2000e-02.


### Setting up dataset and dataloader

In [ ]:
# Alternative class for dataloader
class getC10Pair(CIFAR10):

    def __getitem__(self, idx):

        img, label = self.data[idx], self.targets[idx]
        img = Image.fromarray(img)

        if self.transform is not None:
            img1 = self.transform(img)
            img2 = self.transform(img)

        if self.target_transform is not None:
            label = self.target_transform(label)

        return img1, img2, label

train_data = getC10Pair(root='data', train=True, transform=train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=BATCH, shuffle=True, num_workers=4,
                            drop_last=True)

memory_data = getC10Pair(root='data', train=True, transform=test_transform, download=True)
memory_loader = DataLoader(memory_data, batch_size=BATCH, shuffle=False, num_workers=4)

test_data = getC10Pair(root='data', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=BATCH, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Training

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~KEEP THIS~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~KEEP THIS~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
current_epoch = 0

# DECAY_EPOCHS = 10

results = {'train_loss': [], 'val_loss': [], 'test_acc@1': [], 'test_acc@5': []}
save_name_pre = 'B{}_E{}'.format(BATCH, EPOCHS)

if not os.path.exists('./results'):
    os.mkdir('results')

best_acc = 0.0
for epoch in range(EPOCHS):
    # if epoch % DECAY_EPOCHS == 0 and epoch != 0:
    #     LR = LR * DECAY
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = LR
    #     print("Current learning rate has decayed to %f" %param_group['lr'])

    print("")
    model.train()
    total_loss, total_num, train_bar = 0.0, 0, tqdm(train_loader)

    for x_1, x_2, target in train_bar:
        x_1, x_2 = x_1.cuda(non_blocking=True), x_2.cuda(non_blocking=True)

        feature_1, out_1 = model(x_1)
        feature_2, out_2 = model(x_2)

        loss = criterion(out_1, out_2)
        
        # # [2*B, D]
        # out = torch.cat([out_1, out_2], dim=0)
        # # [2*B, 2*B]
        # sim_matrix = torch.exp(torch.mm(out, out.t().contiguous()) / temperature)
        # mask = (torch.ones_like(sim_matrix) - torch.eye(2 * batch_size, device=sim_matrix.device)).bool()
        # # [2*B, 2*B-1]
        # sim_matrix = sim_matrix.masked_select(mask).view(2 * batch_size, -1)

        # # compute loss
        # pos_sim = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
        # # [2*B]
        # pos_sim = torch.cat([pos_sim, pos_sim], dim=0)
        # loss = (- torch.log(pos_sim / sim_matrix.sum(dim=-1))).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_num += BATCH
        total_loss += loss.item() * BATCH
        train_bar.set_description('Train Epoch: [{}/{}] Loss: {:.4f}'.format(epoch+1, EPOCHS, total_loss / total_num))
        
        train_loss = total_loss/total_num

    results['train_loss'].append(train_loss)

    if epoch < 20:
        warmupscheduler.step()
    if epoch >= 20:
        mainscheduler.step()

    lr = optimizer.param_groups[0]["lr"]

    optimizer.step()

    model.eval()
    total_loss_val, total_top1, total_top5, total_num, feature_bank = 0.0, 0.0, 0.0, 0, []

    with torch.no_grad():
        # generate feature bank
        for data, _, target in tqdm(memory_loader, desc='Feature extracting'):
            feature, out = model(data.cuda(non_blocking=True))
            feature_bank.append(feature)
        # [D, N]
        feature_bank = torch.cat(feature_bank, dim=0).t().contiguous()
        # [N]
        feature_labels = torch.tensor(memory_loader.dataset.targets, device=feature_bank.device)

        # loop test data to predict the label by weighted knn search
        test_bar = tqdm(test_loader)
        for data, data2, target in test_bar:
            data, data2, target = data.cuda(non_blocking=True), data2.cuda(non_blocking=True), target.cuda(non_blocking=True)
            feature, out = model(data)
            feature2, out2 = model(data2)

            total_num += data.size(0)

            loss_val = criterion(out, out2)
            total_loss_val += loss_val.item() * BATCH

            val_loss = total_loss_val/total_num


            # compute cos similarity between each feature vector and feature bank ---> [B, N]
            sim_matrix = torch.mm(feature, feature_bank)
            # [B, K]
            sim_weight, sim_indices = sim_matrix.topk(k=200, dim=-1)
            # [B, K]
            sim_labels = torch.gather(feature_labels.expand(data.size(0), -1), dim=-1, index=sim_indices)
            sim_weight = (sim_weight / TEMP).exp()

            # counts for each class
            one_hot_label = torch.zeros(data.size(0) * 200, 10, device=sim_labels.device)
            # [B*K, C]
            one_hot_label = one_hot_label.scatter(dim=-1, index=sim_labels.view(-1, 1), value=1.0)
            # weighted score ---> [B, C]
            pred_scores = torch.sum(one_hot_label.view(data.size(0), -1, 10) * sim_weight.unsqueeze(dim=-1), dim=1)

            pred_labels = pred_scores.argsort(dim=-1, descending=True)

            total_top1 += torch.sum((pred_labels[:, :1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_top5 += torch.sum((pred_labels[:, :5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            test_bar.set_description('Test Epoch: [{}/{}] Acc@1:{:.2f}% Acc@5:{:.2f}%'
                                     .format(epoch+1, EPOCHS, total_top1 / total_num * 100, total_top5 / total_num * 100))
            
    results['val_loss'].append(val_loss)

    test_acc_1 = total_top1 / total_num * 100
    test_acc_5 = total_top5 / total_num * 100

    results['test_acc@1'].append(test_acc_1)
    results['test_acc@5'].append(test_acc_5)

    # save statistics
    # data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    # data_frame.to_csv('results/{}_statistics.csv'.format(save_name_pre), index_label='epoch')

    if test_acc_1 > best_acc:
        best_acc = test_acc_1
        # torch.save(model.state_dict(), '/workspaces/RepLearning/saved_models/{}_best_model.pth'.format(save_name_pre))
        torch.save(model.state_dict(), './results/{}_lars_adam_with_schedulers_best_model.pth'.format(save_name_pre))
     
    # save_model(model, optimizer, mainscheduler, current_epoch, "SimCLR_CIFAR10_RN20_bestacc.pt")
    # save every epoch as backup
    # torch.save(model.state_dict(), f'./results/backup/{save_name_pre}_e{epoch}.pth')

    # current_epoch+=1

Train Epoch: [1/100] Loss: 6.9873: 100%|██████████| 48/48 [00:23<00:00,  2.06it/s]


Adjusting learning rate of group 0 to 2.4000e-03.


Test Epoch: [1/100] Acc@1:37.41% Acc@5:86.92%: 100%|██████████| 10/10 [00:01<00:00,  6.48it/s]


Train Epoch: [2/100] Loss: 6.7828: 100%|██████████| 48/48 [00:23<00:00,  2.04it/s]


Adjusting learning rate of group 0 to 3.6000e-03.


...

Epoch 00079: adjusting learning rate of group 0 to 1.4293e-02.


Test Epoch: [99/100] Acc@1:69.49% Acc@5:97.89%: 100%|██████████| 10/10 [00:01<00:00,  6.30it/s]


Train Epoch: [100/100] Loss: 6.1970: 100%|██████████| 48/48 [00:23<00:00,  2.07it/s]


Epoch 00080: adjusting learning rate of group 0 to 1.4350e-02.


Test Epoch: [100/100] Acc@1:69.76% Acc@5:98.13%: 100%|██████████| 10/10 [00:01<00:00,  6.53it/s]


In [ ]:
# save statistics
data_frame = pd.DataFrame(data=results, index=range(1, epoch+2))
data_frame.to_csv('results/{}_lars_adam_with_schedulers_statistics.csv'.format(save_name_pre), index_label='epoch')